**Nom Etudiant : Diallo**  

**Prenom Etudiant:Bineta**  

**Classe :ING2GIT** 

**Date limite de dépot : 31/08/2022**


# Travail à Faire:
Télécharger le Datasets sur le lien Drive :  https://drive.google.com/file/d/1tuV0lyb2uWvOInodVTohpdCiAV5FrBX-/view?usp=sharing 

Repondre les questions ci-dessous avec le maximum de precisions et de détails.   
Remplir `FILL_IN` avec les methodes qui correspondent à la réponse adéquate


### Revenus des achats
1. Extraire les revenus d'achat pour chaque événement
2. Filtrer les événements dont le revenu n'est pas nul
3. Vérifiez quels sont les types d'événements qui génèrent des revenus
4. Supprimez la colonne inutile

### Revenus par Traffic
Obtenir les 3 sources de trafic générant le revenu total le plus élevé.
5. Revenus cumulés par source de trafic
7. Obtenir les 3 principales sources de trafic par revenu total
6. Nettoyer les colonnes de revenus pour avoir deux décimales


##### Methods
- DataFrame: `select`, `drop`, `withColumn`, `filter`, `dropDuplicates`,  `groupBy`, `sort`, `limit`
- Column: `isNotNull`, `alias`, `desc`, `cast`, `operators`

In [ ]:
%scala
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder.
                        appName("Session de Rattrapage Big Data").
                        config("spark.ui.port", "0").
                        master("yarn").
                        getOrCreate()

In [ ]:
%scala
val eventsPath = "events.json"

In [ ]:
%scala
val eventsDF = spark.read.json(eventsPath)
eventsDF.show()

### 1. Extraire les revenus d'achat pour chaque événement
Ajouter une nouvelle colonne **`revenue`** en faisant l'extration de **`ecommerce.purchase_revenue_in_usd`**

In [ ]:
%scala
val revenue = eventsDF.select("ecommerce.purchase_revenue_in_usd")
revenue.show()

In [ ]:
import org.apache.spark.sql.functions.col
val revenueDF=eventsDF.withColumn("revenue",col("ecommerce.purchase_revenue_in_usd"))
revenueDF.show()

### 2. Filtrer les événements dont le revenu n'est pas null

In [ ]:
%scala
val purchasesDF = revenueDF.filter("revenue IS NOT NULL")
purchasesDF.show()

### 3. Vérifiez quels sont les types d'événements qui génèrent des revenus
Trouvez des valeurs **`event_name`** uniques dans **`purchasesDF`**. Il y a deux façons de faire :
- Sélectionnez "event_name" et recupérer les enregistrements distincts
- Supprimez les enregistrements en double en fonction de "event_name" uniquement

In [ ]:
%scala
val distinctDF = purchasesDF.select("event_name").distinct()
distinctDF.show

In [ ]:
purchasesDF.dropDuplicates("event_name").show()

### 4. Supprimez la colonne inutile
Puisqu'il n'y a qu'un seul type d'événement, supprimez **`event_name`** de **`purchasesDF`**.

In [ ]:
%scala
val cleanDF = purchasesDF.drop("event_name")
cleanDF.show

### 5. Revenus cumulés par source de trafic
- Grouper par **`traffic_source`**
- Obtenir la somme de **`revenue`** comme **`total_rev`**
- Obtenir la moyenne de **`revenue`** comme **`avg_rev`**

N'oubliez pas d'importer toutes les fonctions intégrées nécessaires.

In [ ]:
%scala
val trafficDF1 = cleanDF.groupBy("traffic_source").sum("revenue")
val trafficDFs=trafficDF1.withColumnRenamed("sum(revenue)","total_rev")
trafficDFs.show()

In [ ]:
val trafficDF2=cleanDF.groupBy("traffic_source").avg("revenue")
val trafficDFss=trafficDF2.withColumnRenamed("avg(revenue)","avg_rev")
trafficDFss.show()

### 2. Obtenir les trois principales sources de trafic par revenu total
- Trier par **`total_rev`** dans l'ordre décroissant
- Se limiter aux trois premières lignes

In [ ]:
%scala
import org.Apache.spark.sql.functions._
val topTrafficDFs = trafficDFs.sort(desc("total_rev"))
topTrafficDFs.show(3)

In [ ]:
val topTrafficDFss = trafficDFss.sort(desc("avg_rev"))
topTrafficDFss.show(3)

### 3. Limitez les colonnes de revenus à deux décimales pointés
- Modifier les colonnes **`avg_rev`** et **`total_rev`** pour les convertir en des nombres avec deux décimales pointés

In [ ]:
%scala
val finalDFs = topTrafficDFs.withColumn("total_rev", col("total_rev").cast("Decimal(10,2)"))
finalDFs.show()

In [ ]:
val finalDFss = topTrafficDFss.withColumn("avg_rev", col("avg_rev").cast("Decimal(10,2)"))
finalDFss.show()